In [ ]:
!pip install flask_ngrok

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 9.1MB/s 
     |████████████████████████████████| 870kB 40.6MB/s 
     |████████████████████████████████| 3.3MB 35.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=36bc3b264e3548cace2f73cc3c274f607bc9309e18d2f09e38e975337ef9a1c4
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
!pip install pytorch-crf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import BertTokenizer
from torchcrf import CRF
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
log_soft = F.log_softmax

class Bert_CRF(nn.Module):
    def __init__(self, bert):
        super(Bert_CRF, self).__init__()
        self.num_labels = num_labels
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, num_labels)
        #self.init_weights()
        self.crf = CRF(num_labels, batch_first=True)    
    
    def forward(self, input_ids, mask, labels=None):  # dont confuse this with _forward_alg above.
        outputs = self.bert(input_ids,attention_mask=mask)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        emission = self.classifier(sequence_output)        
        attn_masks = mask.type(torch.uint8)
        if labels is not None:
            loss = -self.crf(log_soft(emission, 2), labels, mask=attn_masks,reduction='mean')
            return loss
        else:
            prediction = self.crf.decode(emission,mask=attn_masks)
            return prediction

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_version = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_version)
bert_crf_model_loc = '/content/drive/Shareddrives/SWM - NER/models/BERT_CRF/bertCRF.pth'
bert_crf = torch.load(bert_crf_model_loc)
label2id = {'0': 'O',
 '1': 'B-PER',
 '2': 'I-PER',
 '3': 'B-ORG',
 '4': 'I-ORG',
 '5': 'B-LOC',
 '6': 'I-LOC',
 '7': 'B-MISC',
 '8': 'I-MISC'}

In [ ]:
# function to get predictions form brt plus crf model
def bertCRF_pred(data):
    
    
    test_sample = [data.split(" ")]
    sample = tokenizer(test_sample, truncation=True, padding='max_length', is_split_into_words=True)
    s = {}
    s['input_ids'] = sample['input_ids']
    s['mask'] = sample['attention_mask']
    for k, v in s.items():
        s[k] = torch.tensor(s[k])
    batch = { k: v.to(device) for k, v in s.items() }
    outputs = bert_crf(**batch)
    outputs = outputs[0][:len(test_sample[0])]
    #print(outputs)
    t = [label2id[str(i)] for i in outputs]
    if len(test_sample[0]) == len(t):
      print("true")
    else:
      print("false")
    print(t)
    return t

In [ ]:
from collections import OrderedDict
import json
import string
def preds(data):
  sentences = data['data']
  sentences = sentences.translate(str.maketrans('', '', string.punctuation))
  print(sentences)
  sent_words = sentences.split(" ")
  r = bertCRF_pred(sentences)
  retDict = OrderedDict()
  returnList = []
  for i,j in zip(sent_words,r):
    if j!='O':
      returnList.append([i,j])
  newDict = {}
  newDict['count'] = len(returnList)
  newDict['data'] = returnList
  return json.dumps(newDict)


In [ ]:
jsonData = {'data':"New York City, often called simply New York, is the most populous city in the United States. With an estimated 2019 population of 8,336,817 distributed over about 302.6 square miles, New York City is also the most densely populated major city in the United States"}
print(preds(jsonData))

New York City often called simply New York is the most populous city in the United States With an estimated 2019 population of 8336817 distributed over about 3026 square miles New York City is also the most densely populated major city in the United States
true
['B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'B-LOC']
{"count": 12, "data": [["New", "B-LOC"], ["York", "I-LOC"], ["New", "B-LOC"], ["York", "I-LOC"], ["United", "B-LOC"], ["States", "I-LOC"], ["miles", "I-LOC"], ["New", "I-LOC"], ["York", "I-LOC"], ["City", "I-LOC"], ["United", "B-LOC"], ["States", "B-LOC"]]}


In [ ]:
!pip install flask_cors 

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request
from flask_cors import CORS, cross_origin

app = Flask(__name__)
run_with_ngrok(app)
CORS(app)

@app.route('/', methods=['GET', 'POST'])
@cross_origin()
def run_app():
    return "/pred/ for Predictions"


@app.route('/pred/', methods=['GET', 'POST'])
def pred_app():        
    jsonData = request.get_json(force=True)    
    print(jsonData)
    return preds(jsonData)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6f07b0d263a9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Apr/2021 02:51:48] "OPTIONS /pred HTTP/1.1" 308 -
127.0.0.1 - - [06/Apr/2021 02:55:34] "OPTIONS /pred/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 02:55:34] "POST /pred/ HTTP/1.1" 200 -


{'data': 'Jay Buhner hit his 38th homer and Edgar Martinez 23rd for Seattle. The Orioles remained tied with the White Sox American League'}
Jay Buhner hit his 38th homer and Edgar Martinez 23rd for Seattle The Orioles remained tied with the White Sox American League
true
['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'I-MISC']


...

127.0.0.1 - - [06/Apr/2021 03:04:28] "OPTIONS /pred/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 03:04:29] "POST /pred/ HTTP/1.1" 200 -


{'data': 'Jay Buhner hit his 38th homer and Edgar Martinez 23rd for Seattle. The Orioles remained tied with the White Sox American League'}
Jay Buhner hit his 38th homer and Edgar Martinez 23rd for Seattle The Orioles remained tied with the White Sox American League
true
['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'I-MISC']


127.0.0.1 - - [06/Apr/2021 03:04:42] "OPTIONS /pred/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 03:04:43] "POST /pred/ HTTP/1.1" 200 -


{'data': 'Jay Buhner hit his 38th homer and Edgar Martinez 23rd for Seattle. The Orioles remained tied with the White Sox American League'}
Jay Buhner hit his 38th homer and Edgar Martinez 23rd for Seattle The Orioles remained tied with the White Sox American League
true
['B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'I-MISC']


127.0.0.1 - - [06/Apr/2021 03:07:13] "GET /pred HTTP/1.1" 308 -
127.0.0.1 - - [06/Apr/2021 03:07:14] "GET /pred/ HTTP/1.1" 400 -
127.0.0.1 - - [06/Apr/2021 03:31:50] "GET / HTTP/1.1" 200 -
